### Notebook Created and Maintained by Nicolas Tappy. nicolas.tappy@epfl.ch

This Notebook is part of a peer-reviewed scientific publication. If you reuse its content or part of it for scientific analysis, please consider citing it in your references:

https://doi.org/10.1063/5.0080486

# Notebook 04 - Model fitting

This notebook gathers all analysis steps in the analysis of the diamond zoning map, including code to generate plots.

Most plotting commands have been commented out so that the notebook can be run at once without parasitic plots showing up on the screen.

In [1]:
%matplotlib qt
%config Completer.use_jedi = False
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt

### Loading

In [2]:
clmap_ev = hs.load('clmap_IIB-HPHT-diamond_eV_with_variance.hspy')

In [3]:
clmap_ev.metadata

├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2022-04-26T23:05:55.583578+02:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2022-04-26T23:07:48.991821+02:00
│   │   ├── 2
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2022-04-26T23:08:18.166649+02:00
│   │   ├── 3
│   │   │   ├── hyperspy_version = 1.7.0.dev0
│   │   │   ├── io_plugin = hyperspy.io_plugins.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2022-04-26T23:09:21.536595+02:00
│   │   └── 4
│   │       ├── hyperspy_version = 1.7.0.dev0
│   │       ├── io_plugin = hyperspy.io_plugins.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2022-04-26T23:09:53.837005+02:00
│   ├── authors = 
│   ├── date = 
│   ├── original_filename = HYPCard-Bkg500ms-Removed.sur
│   ├── time = 
│   └── title = 
└── Signal
    ├── Noise_properties
    │   ├── Variance_linear_model
    │   │   ├── correlation_factor = 1
    │   │   ├── gain_factor = 1
    │   │   └── gain_offset = 0
    │   └── variance = <Signal1D, title: Variance of , dimensions: (63, 64|875)>
    ├── quantity = CL Intensity (a.u.)
    └── signal_type =

In [4]:
clmap_ev.plot(norm='linear')

c:\users\nicolastappy\documents\git\hyperspy\hyperspy\signal.py:3852: UserWarning: You are summing over an unbinned, non-uniform axis. The result can not be used as an approximation of the integral of the signal. For this functionality, use integrate1D instead.
  warnings.warn("You are summing over an unbinned, non-uniform axis. "


## Mask initialisation

A mask is set to remove dark areas from the fit, where it would not make sense anyways. The data is filtered using booleanisation through comparison

In [5]:
#Integrated signal sub-threshold --> datapoint removed
mask = clmap_ev.integrate1D(-1)<1250

In [6]:
mask.plot()

## Model fitting

A 7 peak model of the data is set up: BXTO,
FXTO,
BXTO+O,
FXTO+O,
FXTA,
BXNP,
BXLO

BXTA can not be detected (oddly enough since FXTA can). It might be simply caught in the tail of the BXTO.

The model is first initialized and the whole spectral axis is removed from the fitting range. Then, with addition of each peak to the model,a custom range typically a few dozens of meV broad is added to the fitting range, and the fit is performed using the bounded, weighted, least squares algorithm. 

Then, the model component is set as inactive and we move onto the next peak.

Finally, the whole dataset is fitted once again with all elements present.

The code is presented with lots of duplicate lines and cells for clarity and ease to display the result during any intermediate step. I am not so dumb that I don't know how to do it with a for loop

In [7]:
#First we create a model object
mod = clmap_ev.create_model()
mod.channel_switches[:] = False

In [9]:
#Model is empty of components
mod.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------

Peak 1: Boron-Bound eXciton + Transverse Optical phonon (BXTO) 

In [10]:
BXTO = hs.model.components1D.Lorentzian()
BXTO.name = 'BXTO'

BXTO.centre.value = 5.219
BXTO.centre.bmin = 5.215
BXTO.centre.bmax = 5.225

BXTO.A.value = 20000
BXTO.A.bmin = 0

BXTO.estimate_parameters(clmap_ev,5.20,5.245)
mod.append(BXTO)
mod.add_signal_range(5.20,5.246)

In [11]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0
    
#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

Peak 2: Free eXciton + Transverse Optical phonon (FXTO) 

In [12]:
FXTO = hs.model.components1D.Lorentzian()
FXTO.name = 'FXTO'
FXTO.centre.value = 5.275
FXTO.centre.bmax = 5.3
FXTO.centre.bmin = 5.25

FXTO.A.value = 2000
FXTO.A.bmin = 0

FXTO.estimate_parameters(clmap_ev,5.25,5.3)
mod.append(FXTO)
mod.add_signal_range(5.25,5.3)

In [13]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0

#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

Peak 3: Boron-Bound eXciton + Transverse Optical + Optical (gamma, 0-momentum) phonon (BXTO+O) recombination

In [14]:
BXTO_O = hs.model.components1D.Lorentzian()
BXTO_O.name = 'BXTO_O'
BXTO_O.centre.value = 5.05
BXTO_O.centre.bmax = 5.08
BXTO_O.centre.bmin = 5.02

BXTO_O.A.value = 2000
BXTO_O.A.bmin = 0

#Sometimes, it helps to set bounds on the FWHM to avoid peak components that "spread to infinity"
BXTO_O.gamma.value = 0.005
BXTO_O.gamma.bmin = 0
BXTO_O.gamma.bmax = 0.03

BXTO_O.estimate_parameters(clmap_ev,5.0,5.1)
mod.append(BXTO_O)
mod.add_signal_range(5.0,5.1)

In [15]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0

#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

Peak 4: Free eXciton + Transverse Optical + Optical (gamma, 0-momentum) phonon (FXTO+O) recombination

In [16]:
FXTO_O = hs.model.components1D.Lorentzian()
FXTO_O.name = 'FXTO_O'

FXTO_O.centre.value = 5.111
FXTO_O.centre.bmin  = 5.10
FXTO_O.centre.bmax  = 5.12

FXTO_O.A.value = 50
FXTO_O.A.bmin = 1
FXTO_O.A.bmax = 150

FXTO_O.gamma.value = 0.019
FXTO_O.gamma.bmin = 0.005
FXTO_O.gamma.bmax = 0.03

FXTO_O.estimate_parameters(clmap_ev,5.0,5.1)
mod.append(FXTO_O)
mod.add_signal_range(5.09,5.16)

In [17]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0

#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

Peak 5: Boron-Bound eXciton, no-phonon (BXNP) recombination

In [18]:
BXNP = hs.model.components1D.Lorentzian()
BXNP.name = 'BXNP'
BXNP.centre.value = 5.36
BXNP.centre.bmax = 5.375
BXNP.centre.bmin = 5.345

BXNP.A.value = 100
BXNP.A.bmin = 0

BXNP.gamma.value = 0.01
BXNP.gamma.bmin = 0
BXNP.gamma.bmax = 0.03

BXNP.estimate_parameters(clmap_ev,5.345,5.375)
mod.append(BXNP)
mod.add_signal_range(5.345,5.375)

In [19]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0

#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

Peak 6: Free eXciton + Transverse Acoustic phonon (FXTA) recombination

In [20]:
FXTA = hs.model.components1D.Lorentzian()
FXTA.name = 'FXTA'
FXTA.centre.value = 5.33
FXTA.centre.bmax = 5.345
FXTA.centre.bmin = 5.315

FXTA.A.value = 10
FXTA.A.bmin = 1
FXTA.A.bmax = 50

FXTA.gamma.value = 0.01
FXTA.gamma.bmin = 0.0012
FXTA.gamma.bmax = 0.03

FXTA.estimate_parameters(clmap_ev,5.305,5.345)
mod.append(FXTA)
mod.add_signal_range(5.305,5.34)

In [21]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0

#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

Peak 7: Boron-Bound eXciton, Longitudinal-optical (BXLO) recombination

In [22]:
BXLO = hs.model.components1D.Lorentzian()
BXLO.name = 'BXLO'
BXLO.centre.value = 5.17
BXLO.centre.bmax = 5.197
BXLO.centre.bmin = 5.160

BXLO.A.value = 100
BXLO.A.bmin = 0
#BXLO.A.bmax = 100
BXLO.A.bmax = 380

BXLO.gamma.value = 0.01
BXLO.gamma.bmin = 0
BXLO.gamma.bmax = 0.06

BXLO.estimate_parameters(clmap_ev,5.155,5.195)
mod.append(BXLO)
mod.add_signal_range(5.155,5.195)

In [23]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

#Resetting the axis index to 0 after the fit is not needed but depending on hyperspy version it allows to avoid a bug
for i in mod.axes_manager.navigation_axes:
    i.index=0

#After fitting we set all parameters to not free
for component in mod:
    for parameter in component.parameters:
        parameter.free = False

  0%|          | 0/3815 [00:00<?, ?it/s]

At the end, the fit is performed once again with all components free to move. 

Warning! The operation be quite time-consuming (40 min on my computer). Recommended 16 Gb of RAM

In [24]:
for component in mod:
    for parameter in component.parameters:
        parameter.free = True

In [25]:
mod.multifit(iterpath='serpentine',bounded=True,mask=mask.data,optimizer='lm',grad='fd')

for i in mod.axes_manager.navigation_axes:
    i.index=0

  0%|          | 0/3815 [00:00<?, ?it/s]

We did not fit the points in the mask but they still were given parameter values from the estimation of parameters. We set them to nan

In [31]:
for component in mod:
    for parameter in component.parameters:
        parameter.map['values'][mask.data] = np.nan
        parameter.map['std'][mask.data] = np.nan

### Storing and saving model

In [27]:
mod.store('Diamond 7 peaks')

In [28]:
clmap_ev.models

└── Diamond_7_peaks
    ├── components
    │   ├── BXLO
    │   ├── BXNP
    │   ├── BXTO
    │   ├── BXTO_O
    │   ├── FXTA
    │   ├── FXTO
    │   └── FXTO_O
    ├── date = 2022-04-26 23:46:50
    └── dimensions = (63, 64|875)

In [29]:
clmap_ev.save("clmap_IIB-HPHT-diamond_eV_with_model.hspy")

### Model plot

In [30]:
#mod.reset_signal_range()
mod.plot()

c:\users\nicolastappy\documents\git\hyperspy\hyperspy\signal.py:3852: UserWarning: You are summing over an unbinned, non-uniform axis. The result can not be used as an approximation of the integral of the signal. For this functionality, use integrate1D instead.
  warnings.warn("You are summing over an unbinned, non-uniform axis. "
